In [5]:
from main import generate_bundle

incoming_data = "data/save_wiki_text/results.txt"

generate_bundle(incoming_data)

['The', 'Arctic', 'hare', '("Lepus', 'arcticus")', 'is', 'a', 'social', 'animal', 'that', 'lives', 'in', 'the', 'frigid', 'tundra', 'of', 'North', 'America,', 'Newfoundland,', 'and', 'Greenland.', 'These', 'hares', 'often', 'congregate', 'in', 'groups', 'of', 'up', 'to', '200', 'individuals.', 'They', 'hop', 'at', 'gr<eat>', 'speeds', 'on', 'their', 'large,', 'powerful', 'hind', 'legs,', 'in', 'a', 'kangaroo-like', 'fashion.', 'The', 'female', 'is', 'called', 'a', 'doe,', 'the', 'male', 'is', 'called', 'a', 'buck,', 'and', 'the', 'baby', 'hare', 'is', 'called', 'a', 'leveret.', 'The', 'young', 'are', 'born', 'open-eyed', 'and', 'furry.', 'Most', 'hares', 'live', 'for', 'about', 'a', 'year', 'in', 'the', 'wild.']
[['The', 'Arctic', 'hare', '("Lepus', 'arcticus")', 'is', 'a', 'social', 'animal', 'that', 'lives', 'in', 'the', 'frigid', 'tundra', 'of', 'North', 'America,', 'Newfoundland,', 'and', 'Greenland.', 'These', 'hares', 'often', 'congregate', 'in', 'groups', 'of', 'up', 'to', '200'